In [ ]:
from helpers import *
from pipeline import *
from visualization import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML

In [ ]:
path = '../../data/080322-ENU-Mouse-Normalisation-Data-Gated.csv'


X, Y, columns_names = load_data(path)

In [ ]:
pipeline = Pipeline(X, Y, 15, channel_names=columns_names)



In [ ]:
pipeline.segment_data()

In [ ]:
pipeline.commit_changes()

In [ ]:
def interact_pick_channel_segment(pipeline, channel):
    channel = columns_names.index(channel)-1
    area_threshold_widget  = widgets.FloatSlider(min=0.0, max=0.05, step=0.001, value=pipeline.area_thresholds[channel], )
    width_threshold_widget = widgets.FloatSlider(min=0.0, max=0.6, step=.025, value=pipeline.width_thresholds[channel])
    depth_threshold_widget = widgets.FloatSlider(min=0.0, max=0.7, step=.025, value=pipeline.depth_thresholds[channel])

    interact(interactive_update_segments,
         area_threshold = area_threshold_widget,
         width_threshold = width_threshold_widget,
         depth_threshold = depth_threshold_widget,
         channel = fixed(channel),
         pipeline=fixed(pipeline))

def interactive_update_segments(pipeline, channel, area_threshold, width_threshold,
                                depth_threshold):

        pipeline.resegment_channel(channel,area_threshold, width_threshold,depth_threshold)


        plot_channel_segments(ch=channel,
                              samples=pipeline.samples,
                               max_per_plt=20,show_labels=False,
                              limit_x0=-.2,limit_x1=1.2)


In [ ]:
ch_widget = widgets.ToggleButtons(
    options=columns_names[1:-2],
    description='Channel',
    disabled=False,
    style={"button_width": "100px"}
)

ch = ch_widget.index

# kde_window_widget = widgets.FloatSlider(min=0.0, max=0.6, step=.025, value=kde_window[ch])
interact(interact_pick_channel_segment,
         channel = ch_widget,
         pipeline=fixed(pipeline))


In [ ]:
def interactive_recompute_and_update_location_hierarchy_and_refs(pipeline, channel, jaccard_thresholds, mode):
    pipeline.recompute_and_update_location_hierarchy_and_refs([channel], jaccard_thresholds)
    plot_gates_dendrogram(pipeline.agg_models_dict[channel],channel,
                          pipeline.jaccard_thresholds[channel], columns_names[channel+1])
    plt.figure()
    plot_gate_as_tiles(pipeline.data_handler, channel, pipeline.Loc_Ref_Dict_All_Ch)
    if mode==1 or mode ==2:
        plt.figure()
        plot_network_graph(channel, pipeline.incidence_matrices[channel], pipeline.channel_names)
        plt.figure()
        sns.heatmap(pipeline.Sim_Matrix_dict[channel], cmap='Blues')
        if mode ==2:
            plot_channel_segments(ch=channel,
                               samples=pipeline.samples,
                               max_per_plt=20,show_labels=False, color_fill_lg= True,
                               limit_x0=-.2,limit_x1=1.2)
def interact_pick_channel_location_grouping(pipeline, channel, mode):
    channel = columns_names.index(channel)-1

    jaccard_threshold_widget = widgets.FloatSlider(min=0.4, max=0.9,
                                                   step=.05, value=pipeline.jaccard_thresholds[channel])


    interact(interactive_recompute_and_update_location_hierarchy_and_refs,
         jaccard_thresholds = jaccard_threshold_widget,
         channel = fixed(channel),
            mode = fixed(mode),
         pipeline=fixed(pipeline))



In [ ]:
pipeline.commit_changes()
ch_widget = widgets.ToggleButtons(
    options=columns_names[1:-2],
    description='Channel',
    disabled=False,
    style={"button_width": "100px"}
)

mode_widget = widgets.Dropdown(
    options=[0, 1, 2],
    value=0,
    description='Mode:',
    disabled=False,
)


interact(interact_pick_channel_location_grouping,
         channel = ch_widget,
         mode = mode_widget,
         pipeline=fixed(pipeline))


In [ ]:
pipeline.Loc_Ref_Dict_All_Ch[0]

In [ ]:
pipeline.commit_changes()
pipeline.update_morphology_hierarchy_and_refs([0,1])

In [ ]:
import time

In [ ]:
time_start = time.time()
pipeline.align_samples([0], verbose=True, subsample_ratio=.5)
time_end = time.time()

In [ ]:
print(time_end-time_start)

In [ ]:
plot_scatter_aligned_vs_original([[1,2]], pipeline, columns_names=pipeline.channel_names)

In [ ]:
print(X.shape)
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# data_hist = np.ones([60, 15, 256],dtype=np.int)
data_hist = {}

In [ ]:
for y in range(60):
    sample_hists = np.ones([15, 256], dtype=np.int)
    for i in range(15):
        x = np.histogram(X[Y==y, i], bins=256)[0]
        sample_hists[i, :] = x
    # data_hist[y, :, :] = sample_hists
    data_hist[y] = sample_hists

In [ ]:
print(data_hist.keys())

In [ ]:
import scipy.io as sio

In [ ]:
save_path = '..\\..\\data\\60_mice_15_channels_hists.mat'
np.savetxt(path,data_hist,delimiter=',',header=columns_names[1:-2],
                   fmt=','.join(['%i'] * 15), comments='')

In [ ]:
df = pd.DataFrame(data_hist.items())

In [ ]:
save_path = '..\\..\\data\\60_mice_15_channels_hists.xlsx'
df.to_excel(save_path)

In [ ]:
df1 = pd.read_excel(save_path)

In [ ]:
print(df1[1][1])